# PepperOpenDayDemo

## Environment Setup

In [1]:
# built-in imports
!pip install requests
!pip install speechrecognition==3.8.1
!pip install paramiko
!pip install pandas

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer main

In [2]:
### Naoqi Related Imports ###
import naoqi
from naoqi import qi
from naoqi import ALProxy

### System Related Imports ###
import fileinput, os, shutil, sys, random, threading, time

### Flask Related Imports ###
import json, requests

### Speech Recognition Imports ###
import pandas as pd
import paramiko
import speech_recognition as sr

/usr/local/lib/python2.7/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography.hazmat.backends import default_backend


In [3]:
### Custom File Handing Imports ###
from fileTransfer import *

### Custom Idle Behaviour Imports ###
from idle import *

### Custom Page Display Imports ###
from displayGeneration import *

### Custom Interaction, Behaviour and Display Imports ###
from interactiveControls import *

### Custom Hard-Coded Prompt Imports ###
from prompts import *

### Main Interaction Imports ###
from humanInteraction import *

## Constants

In [4]:
### Networking Constants ###
GPT_HOST = "10.104.23.208"
PEPPER_HOST = "10.104.23.185"
GPT_PORT = 8891
PEPPER_PORT = 9559

### Audio Processing Constants ###
PEPPER_RECORDINGS_PATH = "/home/nao/microphones/recording.wav"
SERVER_RECORDINGS_PATH = "recordings/recording.wav"

### Display Generation Constants ###
FILE_NAME_TEMP = "display.html"
PEPPER_HTML_PATH = "/home/nao/.local/share/PackageManager/apps/robot-page/html/"  # page.html"
PEPPER_PAGE_LANDING = PEPPER_HTML_PATH + "page.html" # '/home/nao/.local/share/PackageManager/apps/robot-page/html/page.html'
PEPPER_QR_LANDING = PEPPER_HTML_PATH + "webfiles/qr.png"  # switched slash direction
PEPPER_IMG_LANDING = PEPPER_HTML_PATH + "webfiles/img.png"  # switched slash direction
TEXT_BY_ID_PATH = "textbyID.csv"

## MAIN

In [5]:
global idle
global responsesPipeline
global eyes

idle = idling()
eyes = eyesController()

try:
    track_head()

    while True:
        # Empty sentences list
        responsesPipeline = []
        
        # Idle behaviors to keep peppers temperature low
        idle.start_idle_behavior()

        # trigger smart listen
        detect()
        idle.stop()
        
        ## If speech detected record up to 15 seconds of audio
        query = listen()

        # get gpt response
        think(query, responsesPipeline)
    
except KeyboardInterrupt:
    stopListening()

finally:
    stopListening()
    return_to_default_pos()
    idle.stop()
    print('Goodbye')

AttributeError: 'NoneType' object has no attribute 'join'